# Polymers

1 a)

In [3]:
# Importing libraries
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# 1b)Petar lukter

def createPolymer(n):
    """Inputs a number n and returns a horizontal polymer"""
    arr = np.zeros((n,2))
    arr[:,0] = np.linspace(0,n-1,n)
    arr[:,1] = n//2
    print(arr)

[[0. 3.]
 [1. 3.]
 [2. 3.]
 [3. 3.]
 [4. 3.]
 [5. 3.]]


1c)